In [1]:
from __future__ import division
import colorsys
from ipywidgets import interact
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.layouts import column
import numpy as np
from frostsynth.waveform import *
from frostsynth.sampling import *
from frostsynth.ipython import *
output_notebook()

Loading BokehJS ...

In [2]:
x = np.arange(0, 1024) / 1024.0

In [3]:
pw = figure(title="waveform", plot_height=250, plot_width=800, y_range=(-1, 1))
rw = pw.line(x, x)
ps = figure(title="spectrum", plot_height=250, plot_width=800, y_range=(0, 1.2))
rs = ps.vbar(np.arange(128), bottom=0, width=0.6, top=np.zeros(128), color=["#abcdef"] * 128)

In [4]:
def update_plot(y):
    rw.data_source.data["y"] = y
    s = (np.fft.rfft(y) / 256.0)[:128]
    angle = (np.angle(s) + np.pi) / (2 * np.pi)
    rgb = np.array([colorsys.hls_to_rgb(a, 0.4, 0.8) for a in angle]) * 255
    color = ["#%02x%02x%02x" % tuple(c) for c in rgb]
    rs.data_source.data["top"] = abs(s)
    rs.data_source.data["fill_color"] = color
    rs.data_source.data["line_color"] = color
    push_notebook()

In [5]:
def update_fm(amplitude, index_c, index_m, offset):
    y = sine(index_c * x + amplitude * sine(index_m * x) + offset)
    update_plot(y)

In [6]:
show(column(pw, ps), notebook_handle=True);

In [7]:
interact(update_fm, amplitude=(0, 4, 0.05), index_c=(1, 10), index_m=(1, 10), offset=(0, 1, 0.05));

interactive(children=(FloatSlider(value=2.0, description=u'amplitude', max=4.0, step=0.05), IntSlider(value=5, description=u'index_c', max=10, min=1), IntSlider(value=5, description=u'index_m', max=10, min=1), FloatSlider(value=0.0, description=u'offset', max=1.0, step=0.05), Output()), _dom_classes=('widget-interact',))

In [8]:
set_sample_rate(44100)

In [9]:
t = trange(10)
p = 220 * t
env = np.tanh(3 * t) * 0.3

In [10]:
Audio(env * sine(p + sine(p) + t), autoplay=False)